TODO:

In [1]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import to_rgba
import seaborn as sns
from tqdm.notebook import tqdm
from tqdm import trange

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import datasets, transforms
from torchvision.datasets import FashionMNIST

from boring_utils.utils import *

%matplotlib inline 
init_graph()
device = get_device()
set_seed(42, strict=True)

In [2]:
DATASET_PATH = "../data"
CHECKPOINT_PATH = "../model/optm_func/"
os.makedirs(CHECKPOINT_PATH, exist_ok=True)

In [3]:
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.5,), (0.5,))
])

train_set = FashionMNIST(
    root=DATASET_PATH, 
    train=True, 
    download=True, 
    transform=transform
)

test_set = FashionMNIST(
    root=DATASET_PATH, 
    train=False, 
    download=True, 
    transform=transform
)

train_set, val_set = torch.utils.data.random_split(train_set, [50000, 10000])

train_loader = DataLoader(
    train_set, batch_size=1024, shuffle=True, drop_last=False)

val_loader = DataLoader(
    val_set, batch_size=1024, shuffle=True, drop_last=False)

test_loader = DataLoader(
    test_set, batch_size=1024, shuffle=True, drop_last=False)

In [4]:
test_set_no_trans = FashionMNIST(
    root=DATASET_PATH, 
    train=False, 
    download=True, 
    # transform=transform
)

test_loader_no_trans = DataLoader(
    test_set_no_trans, batch_size=1024, shuffle=True, drop_last=False)

Batch Preview

In [5]:
def print_data(dataset, data_loader):
    cprint((dataset.data.float() / 255.0).mean().item())
    cprint((dataset.data.float() / 255.0).std().item())

    imgs, _ = next(iter(data_loader))
    cprint(imgs.mean().item(), imgs.std().item())
    cprint(imgs.max().item(), imgs.min().item())

In [6]:
print_data(test_set, test_loader)

print_data -> (dataset.data.float() / 255.0).mean().item():
0.2868492901325226
print_data -> (dataset.data.float() / 255.0).std().item():
0.3524441719055176
print_data -> imgs.mean().item():
-0.42312583327293396
print_data -> imgs.std().item():
0.7069889307022095
print_data -> imgs.max().item():
1.0
print_data -> imgs.min().item():
-1.0


In [7]:
print_data(test_set_no_trans, test_loader_no_trans)

print_data -> (dataset.data.float() / 255.0).mean().item():
0.2868492901325226
print_data -> (dataset.data.float() / 255.0).std().item():
0.3524441719055176


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'PIL.Image.Image'>

Define some utils

In [ ]:
_get_file_name = lambda model_path, model_name, extension='.tar': os.path.join(model_path, model_name + extension)


def load_model(model_path, model_name, act_fn, net=None, **kargs):
    """
    Loads a saved model from disk.
    """
    model_file = _get_file_name(model_path, model_name)
    if net is None:
        net = BaseNN(act_fn=act_fn, **kargs)
    net.load_state_dict(torch.load(model_file, map_location=device))
    return net


def save_model(model, model_path, model_name):
    """
    Given a model, we save the state_dict and hyperparameters.
    
    Inputs:
        model - Network object to save parameters from
        model_path - Path of the checkpoint directory
        model_name - Name of the model (str)
    """
    os.makedirs(model_path, exist_ok=True)
    model_file = _get_file_name(model_path, model_name)
    torch.save(model.state_dict(), model_file)


def test_model(net, data_loader):
    """
    Test a model on a specified dataset.
    
    Inputs:
        net - Trained model of type BaseNetwork
        data_loader - DataLoader object of the dataset to test on (validation or test)
    """
    net.eval()
    true_preds, count = 0., 0
    for imgs, labels in data_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        with torch.no_grad():
            preds = net(imgs).argmax(dim=-1)
            true_preds += (preds == labels).sum().item()
            count += labels.shape[0]
    test_acc = true_preds / count
    return test_acc 